Exploring Cause of death dataset

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
%matplotlib inline

import matplotlib.pyplot as plt
import seaborn as sb
sb.set()

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [ ]:
death_rate = pd.read_csv('../input/Death_Rates1900-2013.csv')
death_cause = pd.read_csv('../input/leading_cause_death.csv')

In [ ]:
# check if there are any empty or Nan data in the dataset
print(death_rate.isnull().any())
print()
print(death_cause.isnull().any())

In [ ]:
print(death_rate.info())
print()
print(death_cause.info())

In [ ]:
# modify some datasets. delete the formal cause of death data
death_cause.drop(['113_CAUSE_NAME'], axis=1,inplace=True)

In [ ]:
# change the column name capitalization
death_cause.columns = death_cause.columns.str.capitalize()

# ... then change the data type of the DEATHS, ADDR ...
# this throws an error
# investigating shows the presence of non-float values in the two columns
# replace them 
try:
    death_cause.loc[death_cause.Deaths == 'x', 'Deaths'] = 0
except (RuntimeError, TypeError, NameError):
    #print("Oops we may have already removed these {}, {}, {}".format(RuntimeError))
    pass
try:
    death_cause.loc[death_cause.Aadr == '*', 'Aadr'] = 0
except:
    pass
try:
    death_cause.loc[death_cause.Aadr == 'x', 'Aadr'] = 0
except:
    pass
# remove rows where State = United States
# death_cause.groupby('State')['Cause_name'].count() ‐‐ shows values with United States
try:
    death_cause = death_cause[death_cause.State != 'United States']
except:
    pass

#death_cause.Deaths = death_cause.[0 if x == 'x' for x in death_cause.Deaths ]#death_cause.map(lambda x: x=='x' )
# and then convert to numeric values
death_cause['Deaths'] = pd.to_numeric(death_cause['Deaths']) #, errors='coerce')
death_cause['Aadr'] = pd.to_numeric(death_cause['Aadr'])
death_cause.describe()
print()
# run into a provlem with some rows have 'x' and or '*' in 'Deaths' and 'Aadr' columns respectively
#death_cause['Deaths'] = death_cause['Deaths'].map({'x':0})
#death_cause['Aadr'] = death_cause['Aadr'].map({'*':0})
death_cause.info()

In [ ]:
death_cause.describe()

In [ ]:
deaths_by_cause = death_cause.groupby(['Year','Cause_name'])['Deaths'].sum().unstack() * 1/1000000
deaths_by_cause

In [ ]:
#death_cause[death_cause.Cause_name == 'All Causes']
# remove rows where Cause_name = 'All Causes'
idx = death_cause.index[death_cause.Cause_name == 'All Causes']
death_cause.drop(idx, inplace=True)

In [ ]:
#deaths_by_cause = death_cause.groupby(['Year','Cause_name'])['Deaths'].aggregate(['sum']).unstack() * 1/1000 #['min','max', np.mean, np.std])
deaths_by_cause = death_cause.pivot_table('Deaths',index='Year', columns='Cause_name', aggfunc='sum') / 1e6
deaths_by_cause # = deaths_by_cause /1e6

In [ ]:
deaths_by_cause.plot(figsize=(10,7)).legend(loc='center left', bbox_to_anchor=(1, 0.5),
                                            fancybox=True, shadow=True);
plt.ylabel('Total Deaths in millions');
# We can clearly see that Heart Diseases and Cancer are the leading 
# causes of death each year since 2009. 

In [ ]:
deaths_by_cause['Cancer'] / deaths_by_cause
deaths_by_cause = death_cause.pivot_table('Deaths',index='Year', columns='Cause_name',
                                          aggfunc='sum', margins= True) / 1e6
major_causes = 100 * deaths_by_cause[['Cancer','Diseases of Heart']
                                    ].sum(axis=1) /deaths_by_cause[:].sum(axis=1)
major_causes.plot()
plt.ylabel('% of Total Deaths due to Cancer & Heart Disease');
# This shows the sum of leading causes of death decreasing steadily till 2013
# and then rising sharply the next year
# from the previous plot. The recent increases are mostly due to rising Heart Disease

In [ ]:
#with sb.axes_style('white'):
# g = sb.factorplot(x='Cause_Name', y='Deaths', data=death_cause[death_cause.Year!= y], aspect=2,
# hue="Cause_name", kind='bar')
# for each year and cause which state has the highest number of deaths and how many deaths
cause_by_state = death_cause.pivot_table('Deaths',index=['Year','Cause_name'], 
                                         columns='State', aggfunc='sum') / 1e3
#cause_by_state

In [ ]:
zz = cause_by_state.index.levels
#print(death_cause['State'].unique().size)
#max_by_state = np.array((death_cause['State'].unique().size * death_cause['Year'].unique().size, 3))
years = []
cause = []
deaths = []
state =[]
#print(max_by_state.shape)
for yr in zz[0]:
    #print('For year:', y)
    for cs in zz[1]:
        try:
            dths = cause_by_state.loc[yr,cs].max()
            stt = cause_by_state.loc[yr,cs].idxmax()
            years.append(yr)
            cause.append(cs)
            deaths.append(dths*1000)
            state.append(stt)
        except:
            pass
max_cause_deaths = pd.DataFrame({'year':years, 'state':state,
                                'cause_of_max_death':cause, 'deaths_from_cause': deaths})
# would have been nice to have state population data 
max_cause_deaths.query('year == 2004')